In [2]:
%matplotlib inline
import mpld3
mpld3.enable_notebook()
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100

import numpy as np
from random import random, uniform, randrange, triangular
from tqdm import tqdm

In [3]:
#Pytorch imports
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from onnx import optimizer

In [5]:
# base_size = prev_data//group_base
# IMU_size = prev_data//group_IMU
# torque_size = (prev_data + fwd_data)//group_base
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_length, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, output_length),
            nn.Tanh()
        )

        
    def forward(self, x):
        return self.linear_relu_stack(x)

dummy = torch.zeros((1, 270,))
input_var = Variable(dummy)
model = torch.load('model_s.pth')
print(model)
m_inp = torch.ones((1, 270))
print(model(m_inp))

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=270, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=5, bias=True)
    (5): Tanh()
  )
)
tensor([[0.9332, 0.9158, 0.9231, 0.9320, 0.9479]], grad_fn=<TanhBackward>)


In [4]:
from pytorch2keras import pytorch_to_keras
# we should specify shape of the input tensor
k_model = pytorch_to_keras(model, input_var, verbose=False)  
k_model.compile()

In [5]:
from tensorflow import keras
k_model.save('model_s_keras.h5')
# print(k_model.shape)
reconstructed_model = keras.models.load_model("model_s_keras.h5")
print(reconstructed_model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_0 (InputLayer)         [(None, 270)]             0         
_________________________________________________________________
7 (Dense)                    (None, 64)                17344     
_________________________________________________________________
8 (Activation)               (None, 64)                0         
_________________________________________________________________
9 (Dense)                    (None, 32)                2080      
_________________________________________________________________
10 (Activation)              (None, 32)                0         
_________________________________________________________________
11 (Dense)                   (None, 5)                 165       
_________________________________________________________________
output_0 (Activation)        (None, 5)                 0     

In [6]:
import konverter
konverter.konvert('model_s_keras.h5', output_file='export/model_s_np')

AttributeError: 'InputLayer' object has no attribute 'activation'